In [20]:
import numpy as np
import plotly
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from skimage import (filters)

pio.renderers.default='iframe'

In [21]:
def read_full_msi(p: ImzMLParser) -> np.ndarray:
    """

    Function to read a ImzMLParser object into a numpy array

    Args:
        p (ImzMLParser): The ImzMLParser.
    Returns:
        [np.ndarray]: Numpy 3D matrix. y values (axis=0), x values (axis=1), z values (axis=2)

    """
    # Get size of z axis
    max_z = p.mzLengths[0]
    # Get size of y axis
    max_y = p.imzmldict["max count of pixels y"]
    # Get size of x axis
    max_x = p.imzmldict["max count of pixels x"]
    # Create empty numpy 3D matrix
    msi = np.zeros((max_y, max_x, max_z))
    # Loop over each coordinate and add to 3D matrix
    for i, (x, y, _) in tqdm(enumerate(p.coordinates), total=len(p.coordinates), desc="Spectra Loop"):
      # Get mzs and intenisties
      mzs, ints = p.getspectrum(i)
      # Add intenisties to x,y coordinate
      msi[y - 1, x - 1, :] = ints
    return msi

In [27]:
with ImzMLParser("/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed/12-s.imzML") as reader:
  msi = None
  msi = read_full_msi(reader)

Spectra Loop:   0%|          | 0/2665 [00:00<?, ?it/s]

In [28]:
plot_tresh = 95

msi_plot = msi[:, :, (mzs < 1000) & (mzs > 700)].sum(axis=2)

fig = px.imshow(msi_plot, zmin=msi_plot.min(), zmax=np.percentile(msi_plot, plot_tresh), color_continuous_scale='viridis')

fig.update_layout(coloraxis_colorbar=dict(
    tickvals=[msi_plot.min(), np.percentile(msi_plot, plot_tresh)],
    ticktext=["0%", "95%"],
))

fig.show()

In [30]:
with ImzMLParser("/sise/assafzar-group/assafzar/Leor/DHG/Preprocessed/12-s.imzML") as reader:
  print(type(reader.coordinates))

<class 'list'>
